In [1]:
import pandas as pd


df = pd.read_csv('raw_data.csv')
df


,name,min,sec,plus_minus
0,Blake Buchanan,26,17,2
1,Jalen Warley,16,7,0
2,Elijah Saunders,34,31,-13
3,Isaac McKneely,29,48,-11
4,TJ Power,21,7,-18
5,Jacob Cofie,15,45,3
6,Andrew Rohde,23,53,-11
7,Dai Dai Ames,21,37,-11
8,Taine Murray,6,59,1
9,Ishan Sharma,0,57,3


In [2]:
# Clean data
df["name_lower"] = df["name"].str.lower().apply(lambda x: "_".join(x.split()))

# Convert time to minutes
df["time"] = df["min"] + df["sec"] / 60

# Round so that numbers add up to 200
while True:
    df["round_time"] = df["time"].round(0).astype(int)
    if df["round_time"].sum() == 200:
        break
    elif df["round_time"].sum() < 200:
        df["time"] = df["time"] * 1.01
    else:
        df["time"] = df["time"] * 0.99

# Set allowed positions
df["allowed_positions"] = [
    "5",
    "1_3",
    "3_4_5",
    "2",
    "3_4",
    "4_5",
    "1_2_3",
    "1",
    "2_3",
    "2_3",
]

df



,name,min,sec,plus_minus,name_lower,time,round_time,allowed_positions
0,Blake Buchanan,26,17,2,blake_buchanan,26.811628,27,5
1,Jalen Warley,16,7,0,jalen_warley,16.440612,16,1_3
2,Elijah Saunders,34,31,-13,elijah_saunders,35.210452,35,3_4_5
3,Isaac McKneely,29,48,-11,isaac_mckneely,30.398980,30,2
4,TJ Power,21,7,-18,tj_power,21.541112,22,3_4
5,Jacob Cofie,15,45,3,jacob_cofie,16.066575,16,4_5
6,Andrew Rohde,23,53,-11,andrew_rohde,24.363388,24,1_2_3
7,Dai Dai Ames,21,37,-11,dai_dai_ames,22.051162,22,1
8,Taine Murray,6,59,1,taine_murray,7.123698,7,2_3
9,Ishan Sharma,0,57,3,ishan_sharma,0.969095,1,2_3


In [27]:
import numpy as np
from pulp import (
    LpProblem,
    LpVariable,
    lpSum,
    LpInteger,
    LpMinimize,
    LpStatus,
    value,
    PULP_CBC_CMD,
)
import random

# Create binary variables
PLAYERS = df["name_lower"].tolist()
MINS = range(1, 41)
POS = range(1, 6)


def extract_lineup(min: int, choices: dict, pm: dict):
    row = {"min": min}
    for pos in POS:
        for player in PLAYERS:
            if value(choices[player][min][pos]) == 1:
                row[pos] = player
    row["plus_minus"] = value(pm[min])
    return row


def run_opt(seed: int):
    # Create the problem
    prob = LpProblem("gtown_optimize", LpMinimize)

    # choice_<player>_<min>_<pos>
    # E.g., choice_john_doe_1_1 is 1 if John Doe is on the court in minute 1 in position 1
    choices = LpVariable.dicts("choice", (PLAYERS, MINS, POS), cat="Binary")

    # Add plus minus variables
    pm = LpVariable.dicts("pm", (MINS), -10, 10, LpInteger)

    abs_min_changes = LpVariable.dicts("abs_min_changes", (range(1, 40)), lowBound=0)

    # Add a small random coefficient to each choice variable
    random.seed(seed)
    random_coefficients = {}
    for player in PLAYERS:
        for min in MINS:
            for pos in POS:
                # Generate a small random number, e.g., between 0 and 1e-6
                random_coefficients[(player, min, pos)] = random.uniform(0, 1e-6)
                # You can store or print these coefficients if needed

    random_term = lpSum(
        [
            random_coefficients[(player, min, pos)] * choices[player][min][pos]
            for player in PLAYERS
            for min in MINS
            for pos in POS
        ]
    )

    prob += random_term, "sum_random"

    # Set constraints

    # Each player can only be in one position per minute
    for player in PLAYERS:
        for min in MINS:
            prob += lpSum([choices[player][min][pos] for pos in POS]) <= 1

    # Exactly one player per position per minute
    for min in MINS:
        for pos in POS:
            prob += lpSum([choices[player][min][pos] for player in PLAYERS]) == 1
            for player in PLAYERS:
                if min < 40:
                    prob += (
                        abs_min_changes[min]
                        >= choices[player][min + 1][pos] - choices[player][min][pos]
                    )

    for player in PLAYERS:
        # Player minutes have to match data
        total_min = df.loc[df["name_lower"] == player, "round_time"].iloc[0]
        prob += (
            lpSum([choices[player][min][pos] for min in MINS for pos in POS])
            == total_min
        )

        # Set specific player position constraints
        allowed_positions = (
            df.loc[df["name_lower"] == player, "allowed_positions"].iloc[0].split("_")
        )
        for pos in POS:
            if str(pos) not in allowed_positions:
                prob += lpSum([choices[player][min][pos] for min in MINS]) == 0

        # Set plus minus constraint
        player_pm = df.loc[df["name_lower"] == player, "plus_minus"].iloc[0]

        # Create binary variables to track if player is on court each minute
        on_court = LpVariable.dicts(f"on_court_{player}", MINS, cat="Binary")

        # Create variables for player's contribution to plus-minus each minute
        player_pm_per_min = LpVariable.dicts(
            f"pm_{player}", MINS, None, None, LpInteger
        )

        # Link on_court variable to choices
        for min in MINS:
            # Track if player is on court
            prob += on_court[min] == lpSum([choices[player][min][pos] for pos in POS])

            # If player is on court (on_court[min] = 1), then player_pm_per_min[min] = pm[min]
            # If player is off court (on_court[min] = 0), then player_pm_per_min[min] = 0
            M = 100  # A large number that's bigger than any possible plus-minus value
            prob += player_pm_per_min[min] <= pm[min] + M * (1 - on_court[min])
            prob += player_pm_per_min[min] >= pm[min] - M * (1 - on_court[min])
            prob += player_pm_per_min[min] <= M * on_court[min]
            prob += player_pm_per_min[min] >= -M * on_court[min]

        # Sum of player's plus-minus contributions must equal their total plus-minus
        prob += lpSum([player_pm_per_min[min] for min in MINS]) == player_pm

    # Overall plus minus constraint
    prob += lpSum([pm[min] for min in MINS]) == -11

    # Write the problem
    prob.writeLP("gtown_optimize.lp")

    # Solve
    prob.solve(
        PULP_CBC_CMD(
            msg=True,
            timeLimit=10,  # Allow up to 5 minutes
        )
    )

    print("Status:", LpStatus[prob.status])
    print(prob.objective.value())

    rows = []
    for min in MINS:
        rows.append(extract_lineup(min, choices, pm))

    if prob.status != 1:
        return None
    else:
        return pd.DataFrame(rows)

In [28]:
from tqdm import tqdm


all_lineup_dfs = []
tt = 23
for i in tqdm(range(tt, tt + 1)):
    lineup_df = run_opt(i)
    if lineup_df is not None:
        all_lineup_dfs.append(lineup_df.assign(seed=i))
    else:
        print(f"Failed on seed {i}")

lineup_df = pd.concat(all_lineup_dfs)



  0%|          | 0/1 [00:00<?, ?it/s]

In [11]:
lineup_df

,min,1,2,3,4,5,plus_minus,seed
0,1,dai_dai_ames,isaac_mckneely,andrew_rohde,tj_power,blake_buchanan,-3.0,23
1,2,dai_dai_ames,andrew_rohde,elijah_saunders,jacob_cofie,blake_buchanan,0.0,23
2,3,dai_dai_ames,isaac_mckneely,jalen_warley,tj_power,elijah_saunders,-3.0,23
3,4,dai_dai_ames,isaac_mckneely,tj_power,elijah_saunders,blake_buchanan,-3.0,23
4,5,andrew_rohde,isaac_mckneely,jalen_warley,elijah_saunders,jacob_cofie,2.0,23
5,6,dai_dai_ames,isaac_mckneely,elijah_saunders,jacob_cofie,blake_buchanan,3.0,23
6,7,andrew_rohde,isaac_mckneely,tj_power,elijah_saunders,blake_buchanan,-3.0,23
7,8,andrew_rohde,isaac_mckneely,tj_power,elijah_saunders,blake_buchanan,3.0,23
8,9,jalen_warley,isaac_mckneely,andrew_rohde,jacob_cofie,blake_buchanan,3.0,23
9,10,dai_dai_ames,isaac_mckneely,jalen_warley,jacob_cofie,elijah_saunders,-3.0,23


In [7]:
lineup_df


,min,1,2,3,4,5,plus_minus,seed
0,1,dai_dai_ames,ishan_sharma,elijah_saunders,jacob_cofie,blake_buchanan,3.0,1
1,2,dai_dai_ames,isaac_mckneely,jalen_warley,tj_power,jacob_cofie,-1.0,1
2,3,andrew_rohde,isaac_mckneely,tj_power,elijah_saunders,jacob_cofie,3.0,1
3,4,dai_dai_ames,isaac_mckneely,taine_murray,elijah_saunders,blake_buchanan,-3.0,1
4,5,dai_dai_ames,isaac_mckneely,jalen_warley,elijah_saunders,blake_buchanan,-3.0,1
...,...,...,...,...,...,...,...,...
35,36,dai_dai_ames,isaac_mckneely,andrew_rohde,elijah_saunders,blake_buchanan,-3.0,99
36,37,andrew_rohde,taine_murray,tj_power,elijah_saunders,blake_buchanan,-3.0,99
37,38,dai_dai_ames,taine_murray,jalen_warley,elijah_saunders,blake_buchanan,-3.0,99
38,39,jalen_warley,isaac_mckneely,andrew_rohde,elijah_saunders,blake_buchanan,3.0,99


In [12]:
long = lineup_df.melt(id_vars=["min", "plus_minus", "seed"], value_vars=list(POS), var_name="position", value_name="player")

# Fill in missing values
long["minutes"] = 1
all_vals = long[["position", "player"]].drop_duplicates()
all_vals["minutes"] = 0
all_vals["plus_minus"] = 0
all_vals = pd.concat([all_vals.assign(seed=seed) for seed in lineup_df.seed.unique()], ignore_index=True)
long = pd.concat([long, all_vals], ignore_index=True)

tmp = long.groupby(["position", "seed", "player"]).agg(
    minutes=("minutes", "sum"),
    plus_minus=("plus_minus", "sum"),
).reset_index()
tmp["plus_minus"] = tmp["plus_minus"].astype(int)
tmp2 = tmp.groupby(["position", "player"]).agg(
    min_minutes=("minutes", "min"),
    max_minutes=("minutes", "max"),
    avg_minutes=("minutes", "mean"),
    pct5_minutes=("minutes", lambda x: x.quantile(0.05)),
    pct95_minutes=("minutes", lambda x: x.quantile(0.95)),
    min_plus_minus=("plus_minus", "min"),
    max_plus_minus=("plus_minus", "max"),
    avg_plus_minus=("plus_minus", "mean"),
    pct5_plus_minus=("plus_minus", lambda x: x.quantile(0.05)),
    pct95_plus_minus=("plus_minus", lambda x: x.quantile(0.95)),
).reset_index().sort_values(["player", "position"])
display(tmp2)


,position,player,min_minutes,max_minutes,avg_minutes,pct5_minutes,pct95_minutes,min_plus_minus,max_plus_minus,avg_plus_minus,pct5_plus_minus,pct95_plus_minus
0,1,andrew_rohde,7,13,10.166667,8.00,12.00,-14,7,-2.106061,-9.00,4.75
3,2,andrew_rohde,3,7,5.181818,4.25,7.00,-13,3,-3.787879,-9.00,1.00
7,3,andrew_rohde,5,12,8.651515,6.00,11.00,-15,10,-5.106061,-12.00,2.75
16,5,blake_buchanan,27,27,27.000000,27.00,27.00,2,2,2.000000,2.00,2.00
1,1,dai_dai_ames,22,22,22.000000,22.00,22.00,-11,-11,-11.000000,-11.00,-11.00
8,3,elijah_saunders,3,13,9.772727,5.00,12.00,-16,12,-0.469697,-8.75,9.00
13,4,elijah_saunders,15,29,20.333333,18.00,26.75,-21,8,-7.030303,-17.75,4.75
17,5,elijah_saunders,3,7,4.893939,3.00,6.00,-17,3,-5.500000,-11.75,0.00
4,2,isaac_mckneely,30,30,30.000000,30.00,30.00,-11,-11,-11.000000,-11.00,-11.00
5,2,ishan_sharma,0,1,0.939394,0.25,1.00,0,3,2.818182,0.75,3.00


In [13]:
tmp.query("player == 'elijah_saunders' & position == 3 & minutes == 3")


,position,seed,player,minutes,plus_minus
673,3,52,elijah_saunders,3,3


In [15]:
tmp.query("seed == 52")


,position,seed,player,minutes,plus_minus
105,1,52,andrew_rohde,11,-9
106,1,52,dai_dai_ames,22,-11
107,1,52,jalen_warley,7,9
338,2,52,andrew_rohde,7,0
339,2,52,isaac_mckneely,30,-11
340,2,52,ishan_sharma,1,3
341,2,52,taine_murray,2,-3
672,3,52,andrew_rohde,6,-2
673,3,52,elijah_saunders,3,3
674,3,52,ishan_sharma,0,0


In [14]:
lineup_df.query("seed == 52")

,min,1,2,3,4,5,plus_minus,seed
0,1,jalen_warley,ishan_sharma,tj_power,elijah_saunders,blake_buchanan,3.0,52
1,2,dai_dai_ames,taine_murray,jalen_warley,tj_power,jacob_cofie,-3.0,52
2,3,dai_dai_ames,isaac_mckneely,andrew_rohde,tj_power,jacob_cofie,0.0,52
3,4,dai_dai_ames,isaac_mckneely,taine_murray,tj_power,elijah_saunders,-3.0,52
4,5,dai_dai_ames,isaac_mckneely,tj_power,elijah_saunders,blake_buchanan,-2.0,52
5,6,dai_dai_ames,isaac_mckneely,andrew_rohde,elijah_saunders,blake_buchanan,3.0,52
6,7,dai_dai_ames,isaac_mckneely,jalen_warley,jacob_cofie,blake_buchanan,3.0,52
7,8,jalen_warley,isaac_mckneely,tj_power,elijah_saunders,blake_buchanan,-3.0,52
8,9,dai_dai_ames,taine_murray,andrew_rohde,elijah_saunders,blake_buchanan,0.0,52
9,10,andrew_rohde,isaac_mckneely,taine_murray,elijah_saunders,blake_buchanan,3.0,52


In [20]:
tmp2.to_csv("solution.csv", index=False)